### 1. Connect to the Azure Quantum workspace

To connect to the Azure Quantum service, call the `connect` function as seen below. This can take 30 seconds to run the first time, as the Q# environment is initializing.

In [30]:
import qsharp.azure

targets = qsharp.azure.connect(
            resourceId = "/subscriptions/7a4782f6-5786-48b6-9067-dcce8a1616bb/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/QSWfinal",
            location = "eastus")



Connecting to Azure Quantum...

Authenticated using Microsoft.Azure.Quantum.Authentication.TokenFileCredential


Connected to Azure Quantum workspace QSWfinal in location eastus.


We can use the resulting object to see which _targets_ are available for submission.

In [31]:
import qsharp

print("This workspace's targets:")
for target in targets:
    print("-", target.id)


This workspace's targets:
- ionq.qpu
- ionq.qpu.aria-1
- ionq.simulator
- microsoft.estimator
- quantinuum.hqs-lt-s1
- quantinuum.hqs-lt-s1-apival
- quantinuum.hqs-lt-s2
- quantinuum.hqs-lt-s2-apival
- quantinuum.hqs-lt-s1-sim
- quantinuum.hqs-lt-s2-sim
- quantinuum.qpu.h1-1
- quantinuum.sim.h1-1sc
- quantinuum.qpu.h1-2
- quantinuum.sim.h1-2sc
- quantinuum.sim.h1-1e
- quantinuum.sim.h1-2e
- rigetti.sim.qvm
- rigetti.qpu.aspen-m-2
- rigetti.qpu.aspen-m-3


### ❕ Do you see `ionq.simulator` in your list of targets? If so, you're ready to keep going.

Don't see it? You may need to add IonQ to your workspace to run this sample. Navigate to the **Providers** page in the portal and click **+Add** to add the IonQ provider. Don't worry, there's a free credits plan available.

## IonQ: The quantum provider
Azure Quantum partners with third-party companies to deliver solutions to quantum jobs. These company offerings are called *providers*. Each provider can offer multiple *targets* with different capabilities. See the table below for IonQ's targets.

| Target name | Target ID | Number of qubits | Description |
| --- | --- | --- | --- |
| Quantum simulator | `ionq.simulator` | 29 qubits | IonQ's cloud-based idealized simulator. Free of cost. |
| Aria 1 | `ionq.qpu.aria-1` | 23 qubits | IonQ's Aria 1 trapped-ion quantum computer. This is real quantum hardware, not a simulation. |
| Quantum computer | `ionq.qpu` | 11 qubits | IonQ's trapped-ion quantum computer. This is real quantum hardware, not a simulation. |

[documentation](https://docs.microsoft.com/azure/quantum/provider-ionq).

### 2. Build the quantum program
Quantum Phase Estimation

In [32]:
%%qsharp


open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

operation QPE(numCountingBits : Int, eigenVector : Qubit[], 
                oracle : (Qubit[]) => Unit is Adj + Ctl) : Result[] {
    // prepare input register
    use input = Qubit[numCountingBits];
    ApplyToEach(H, input);
    // apply U^(2^t-1), ..., U^(2^0)
    for i in 0 .. numCountingBits - 1 {
        let powered = 2 ^ (numCountingBits - 1 - i);
        for j in 0 .. powered - 1 {
            Controlled oracle([input[i]], eigenVector);
        }
    }
    Adjoint Lab8_QFT(BigEndian(input));
    return MultiM(input);
}

operation Lab3_swap(register : Qubit[]) : Unit is Adj + Ctl {
    let len = Length(register) - 1;
    if (len > 0) {
        let lenHalf = len / 2;
        for i in 0 .. lenHalf {
            if (i != len - i) {
                SWAP(register[i], register[len - i]);
            }
        }
    }
}

operation Lab8_QFT (register : BigEndian) : Unit is Adj + Ctl {
    let len = Length(register!);
    for i in 0 .. len - 1 {
        H(register![i]);

        for j in i + 1 .. len - 1 {
            Controlled Microsoft.Quantum.Intrinsic.R1Frac([register![j]], 
                                                (1, j - i, register![i]));
        }
    }
    Lab3_swap(register!);
}

operation testQPE_T() : Result[] {
    use target = Qubit();
    X(target);      // prepare the eigenvector |1>
    return QPE(3, [target], T_wrapper);
}

// wrapper for T gate, in order to match type
operation T_wrapper(target : Qubit[]) : Unit is Adj + Ctl {
    T(target[0]);   // T-gate: e^2πi/8 phase rotation
}


### 3. Submit the quantum program to IonQ

We will use the `execute` function to run the quantum program above on IonQ's `ionq.simulator` target. 

In [33]:
# Set the target to ionq.qpu.aria-1
qsharp.azure.target("ionq.qpu")

# We'll use 20 shots. Timeout is in seconds.
result = qsharp.azure.execute(testQPE_T, shots=1, jobName="qpe", timeout=700)


Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu
Submitting testQPE_T to target ionq.qpu...
Job successfully submitted.
   Job name: qpe
   Job ID: b9ba25a8-afd2-4164-95e2-b09ce37bf582
Waiting up to 700 seconds for Azure Quantum job to complete...
[16:53:24] Current job status: Waiting
[16:53:29] Current job status: Waiting
[16:53:34] Current job status: Waiting
[16:53:39] Current job status: Waiting
[16:53:44] Current job status: Waiting
[16:53:49] Current job status: Waiting
[16:53:54] Current job status: Waiting
[16:53:59] Current job status: Waiting
[16:54:04] Current job status: Waiting
[16:54:09] Current job status: Waiting
[16:54:14] Current job status: Waiting
[16:54:19] Current job status: Waiting
[16:54:24] Current job status: Waiting
[16:54:29] Current job status: Waiting
[16:54:35] Current job status: Waiting
[16:54:40] Current job status: Waiting
[16:54:45] Current job status: Waiting
[16:54:50] Current job status: Waiting


TimeoutError: Timeout waiting for output

The job ID can be used to retrieve the results later using the `output` and `status` functions or by viewing it under the **Job management** section of the portal.

### 4. Visualize job results 
You can also view a histogram of the results using [`pyplot`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html):

In [ ]:
from matplotlib import pyplot

pyplot.bar(result.keys(), result.values())
pyplot.title("Result")
pyplot.xlabel("Measurement")
pyplot.ylabel("Probability")

pyplot.show()
